In [1]:
clear

In [2]:
import os

In [3]:
os.chdir("/Users/chris/Desktop/Dropbox/MasterThesisAFM/Coding")
os.getcwd()

'/Users/chris/Desktop/Dropbox/MasterThesisAFM/Coding'

In [4]:
import csv

In [5]:
#CREATE DICTIONARY FOR NODES
def reader(filename):
    for (lineno, line) in enumerate(open(filename)):
        if lineno > 0: # skip header
            yield line

filename = "case1_nodes.csv"
(l_nodes, nodenames, beg, end, change) = zip(*( row for row in csv.reader(reader(filename))))

d_nodes = dict(zip(l_nodes, change))

#for k, v in d_nodes.items():
    #print(k," ",v)



In [6]:
#{k:int(v) for k, v in dict_nodes.items()} #dictionary comprehension if change needed

In [7]:
import networkx as nx 
DG=nx.DiGraph()

In [8]:
#ADD SOURCE/SINK EDGES, by definition all nodes are included in edgelist
for k, v in d_nodes.items():
    
    if int(v)<0:
        DG.add_edge('S',k)
        nx.set_edge_attributes(DG, 'capacity', {('S',k): abs(int(v))})
        print("from 'S' to", k, "with capacity =",abs(int(v)))

    if int(v)>0:
        DG.add_edge(k,"T")
        nx.set_edge_attributes(DG, 'capacity', {(k, 'T'): abs(int(v))})
        print("from", k,"to 'T'","with capacity =",int(v))

from A8 to 'T' with capacity = 45
from 'S' to A4 with capacity = 30
from A12 to 'T' with capacity = 180
from 'S' to A3 with capacity = 35
from A2 to 'T' with capacity = 130
from A13 to 'T' with capacity = 50
from 'S' to A7 with capacity = 75
from 'S' to A9 with capacity = 15
from 'S' to A5 with capacity = 5
from 'S' to A11 with capacity = 270
from A1 to 'T' with capacity = 45
from 'S' to A10 with capacity = 20
from 'S' to A6 with capacity = 10
from A14 to 'T' with capacity = 10


In [9]:
#nx.edges(DG) #check edges that for source/sink nodes

In [10]:
#ADD TRANSACTION EDGES
def reader(filename2):
    for (lineno, line) in enumerate(open(filename2)):
        if lineno > 0: # skip header
            yield line

filename2 = "case1_edges.csv"
(from_, to_) = zip(*( row for row in csv.reader(reader(filename2))))

l_edges = list(zip(from_, to_))

for f,t in l_edges:
    DG.add_edge(f,t)
    DG.edge[f][t]['capacity'] = 99999
  

In [11]:
nx.edges(DG) #check edges

[('A7', 'A5'),
 ('A7', 'A3'),
 ('A4', 'A8'),
 ('A1', 'A7'),
 ('A1', 'A3'),
 ('A1', 'A10'),
 ('A1', 'T'),
 ('A1', 'A8'),
 ('A12', 'T'),
 ('A3', 'A12'),
 ('A2', 'A1'),
 ('A2', 'T'),
 ('A13', 'T'),
 ('A8', 'A14'),
 ('A8', 'T'),
 ('A9', 'A13'),
 ('S', 'A7'),
 ('S', 'A4'),
 ('S', 'A9'),
 ('S', 'A5'),
 ('S', 'A11'),
 ('S', 'A6'),
 ('S', 'A10'),
 ('S', 'A3'),
 ('A11', 'A2'),
 ('A5', 'A13'),
 ('A10', 'A1'),
 ('A6', 'A13'),
 ('A14', 'T')]

In [12]:
print('no. of nodes:',DG.number_of_nodes())
print('no. of edges:',DG.number_of_edges())
print('maximum flow:',nx.maximum_flow_value(DG, s='S', t='T'))
print('minimum cut: ',nx.minimum_cut_value(DG, s="S", t="T"))

no. of nodes: 16
no. of edges: 29
maximum flow: 460
minimum cut:  460


In [13]:
#run max-flow min cut algorithm
data = nx.maximum_flow(DG, s="S", t="T")
nx.maximum_flow(DG, s="S", t="T")

(460,
 {'A1': {'A10': 0, 'A3': 90, 'A7': 0, 'A8': 25, 'T': 45},
  'A10': {'A1': 20},
  'A11': {'A2': 270},
  'A12': {'T': 180},
  'A13': {'T': 50},
  'A14': {'T': 10},
  'A2': {'A1': 140, 'T': 130},
  'A3': {'A12': 180},
  'A4': {'A8': 30},
  'A5': {'A13': 25},
  'A6': {'A13': 10},
  'A7': {'A3': 55, 'A5': 20},
  'A8': {'A14': 10, 'T': 45},
  'A9': {'A13': 15},
  'S': {'A10': 20,
   'A11': 270,
   'A3': 35,
   'A4': 30,
   'A5': 5,
   'A6': 10,
   'A7': 75,
   'A9': 15},
  'T': {}})

In [14]:
y1 = []

In [15]:
for x, z in l_edges:
    try:
        y1.append(data[1][x][z])
    except:
        pass

In [16]:
import numpy as np
print(np.__version__)



1.11.1


In [17]:
y = np.array(y1).reshape(len(l_edges),1)
print(y)

[[270]
 [140]
 [ 55]
 [ 90]
 [180]
 [ 30]
 [ 20]
 [ 25]
 [ 10]
 [  0]
 [ 15]
 [  0]
 [ 20]
 [ 10]
 [ 25]]


***DERIVE INCIDENCE MATRIX***

In [18]:
def incidence_matrix(A, nodelist=None, edgelist=None, 
                     oriented=True, weight=None):
    if nodelist is None:
        nodelist = l_nodes
    if edgelist is None:
        edgelist = l_edges
    A = np.zeros((len(nodelist),len(edgelist)))
    node_index = dict( (node,i) for i,node in enumerate(nodelist) )
    for ei,e in enumerate(edgelist):
        (u,v) = e[:2]
        if u == v: continue  # self loops give zero column
        try:
            ui = node_index[u]
            vi = node_index[v]
        except KeyError:
            raise NetworkXError('node %s or %s in edgelist '
                                'but not in nodelist"%(u,v)')
        if weight is None:
            wt = 1
        else:
            if DG.is_multigraph():
                ekey = e[2]
                wt = DG[u][v][ekey].get(weight,1)
            else:
                wt = DG[u][v].get(weight,1)
        if oriented:
            A[ui,ei] = -wt
            A[vi,ei] = wt
        else:
            A[ui,ei] = wt
            A[vi,ei] = wt
    return np.asmatrix(A)

In [19]:
A = incidence_matrix("A")
print(A)

[[ 0.  1.  0. -1.  0.  0.  0.  0.  0. -1.  0. -1.  1.  0. -1.]
 [ 1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0. -1.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0. -1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -1.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]]


***PERFORM CONSISTENCY CHECK***

In [20]:
#derive vector x
int_change = []
for value in change:
    int_change.append(int(value))
    
x = np.array(int_change).reshape(len(l_nodes), 1)
print(x)

[[  45]
 [ 130]
 [ -35]
 [ -30]
 [  -5]
 [ -10]
 [ -75]
 [  45]
 [ -15]
 [ -20]
 [-270]
 [ 180]
 [  50]
 [  10]]


In [24]:
#check if Ay = x relationship holds
Ay = incidence_matrix("A")*y
Ay.astype(int)
print(Ay)

[[  45.]
 [ 130.]
 [ -35.]
 [ -30.]
 [  -5.]
 [ -10.]
 [ -75.]
 [  45.]
 [ -15.]
 [ -20.]
 [-270.]
 [ 180.]
 [  50.]
 [  10.]]


In [25]:
#test = np.subtract(Ay, x)
#print(test)

In [26]:
if x.all == Ay.all:
    print("consistent y")
else:
    print("inconsistent y")

inconsistent y
